<a href="https://colab.research.google.com/github/alihammadbaig/Sentiment_Analysis/blob/master/Sentiment_Analyzer_using_Python_and_Google_NL_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**I'm going to make a Telegram Bot that will do the sentiment analysis of tweets related to the keyword that we define.**

# Libraries


*   [Tweepy](http://www.tweepy.org/) to gather tweet data
*   [nltk](https://www.nltk.org/) to cleanse the tweets
*  [Google Natural Language API](https://cloud.google.com/natural-language/) for sentiment analysis
*  [Python Telegram Bot](https://github.com/python-telegram-bot/python-telegram-bot) to send the results through Telegram chat



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls "/content/gdrive/My Drive/Colab Notebooks/data/"

In [0]:
!pip install tweepy
!pip install nltk
!pip install google-cloud-language
!pip install python-telegram-bot

In [0]:
!export GOOGLE_APPLICATION_CREDENTIALS="/content/gdrive/My Drive/Colab Notebooks/data/creds.json"

# Plan of Action

**This program will gather all the tweets containing the defined keyword in the last 24 hours with a maximum of 50 tweets. Then it will analyze the tweets’ sentiments one by one. We will send the result (average sentiment score) through Telegram chat.**

This is a simple workflow of our program.

connect to the Twitter API -> search tweets based on the keyword -> clean all of the tweets -> get tweet’s sentiment score -> send the result

## 1-Connect to the Twitter API

In [0]:
import tweepy

In [0]:
ACC_TOKEN = '13933272-NZy9z26o4uMxRcCE0Z2eUM5RUHm5o7Q83S5uMhzI0'
ACC_SECRET = 'NoTccC3XcaWxhTOvn5IaLDMrH2lhQS4iZ7bH2n0OKl0f0'
CONS_KEY = 'TnFp4Au3MUuTTqNxSrtEz93Nq'
CONS_SECRET = 'EPg80LOisTbmNEw3B967COugeQobo6BTBpr6M5BnLueFcHc2N2'

### API Authentication

In [0]:
def authentication(cons_key, cons_secret, acc_token, acc_secret):
    auth = tweepy.OAuthHandler(cons_key, cons_secret)
    auth.set_access_token(acc_token, acc_secret)
    api = tweepy.API(auth)
    return api

### Search the Tweets

In [0]:
def search_tweets(keyword, total_tweets):
    # gather the tweets from the last 24 hours
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    
    api = authentication(CONS_KEY,CONS_SECRET,ACC_TOKEN,ACC_SECRET)
    
    search_result = tweepy.Cursor(api.search, 
                                  q=keyword, 
                                  since=yesterday_date, 
                                  result_type='recent', 
                                  lang='en').items(total_tweets)
    
    return search_result